# All Models Fusion

## Set up

###  Install required packages.

In [29]:
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['PYTHONWARNINGS'] = "ignore"
print(torch.__version__)
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

2.6.0+cu124
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-7b2qek49
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-7b2qek49
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 4410298df8de8ca9f13ba4ee12e6a30ec82c42d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Unzip Data and Utils

In [2]:
!unzip "utils.zip" -d '.'

Archive:  utils.zip
   creating: ./utils/
   creating: ./utils/__pycache__/
  inflating: ./__MACOSX/utils/.___pycache__  
  inflating: ./utils/threshold.py    
  inflating: ./utils/comparing.py    
  inflating: ./utils/__pycache__/comparing.cpython-312.pyc  
  inflating: ./__MACOSX/utils/__pycache__/._comparing.cpython-312.pyc  
  inflating: ./utils/__pycache__/threshold.cpython-312.pyc  
  inflating: ./__MACOSX/utils/__pycache__/._threshold.cpython-312.pyc  


In [3]:
!unzip "data.zip" -d '.'

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
  inflating: ./__MACOSX/data/external/rphunter/Dataset/Incidents-Source-Code/2024-5-2-NovaMind/._@openzeppelin  
  inflating: ./data/external/rphunter/Dataset/Incidents-Source-Code/2021-11-25-Sloth Inu/0x85bcaee8befa419becb90e8b405d131ee9e31877.sol  
  inflating: ./__MACOSX/data/external/rphunter/Dataset/Incidents-Source-Code/2021-11-25-Sloth Inu/._0x85bcaee8befa419becb90e8b405d131ee9e31877.sol  
  inflating: ./data/external/rphunter/Dataset/Incidents-Source-Code/2021-8-26-Snakedefi/0xf882413338a88F3d41f0df4c31d8CA5d33e3e3d1.sol  
  inflating: ./__MACOSX/data/external/rphunter/Dataset/Incidents-Source-Code/2021-8-26-Snakedefi/._0xf882413338a88F3d41f0df4c31d8CA5d33e3e3d1.sol  
  inflating: ./data/external/rphunter/Dataset/Incidents-Source-Code/2022-9-22-SiriusToken/0x9bc699780A326C059C660C6ee2EF1D8583D01aEa.sol  
  inflating: ./__MACOSX/data/external/rphunter/Dataset/Incidents-Source-Code/2022-9-22-SiriusToken/._0x9bc699780A326C05

### Import and Var

In [81]:
import warnings
warnings.filterwarnings("ignore")

In [82]:
import os
import pandas as pd
from pathlib import Path
import json
import pickle
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch_geometric.utils import from_networkx

from utils.comparing import report_for_multiple_model_as_same_set, evaluate_multilabel_classification
from utils.threshold import tune_thresholds

PATH = './data/labeled'

## Load Data

In [83]:
def load_feature(file):
    df = pd.read_csv(os.path.join(PATH, file), index_col=0)
    df.index = df.index.str.lower()
    return df

In [84]:
y = load_feature('groundtruth.csv')
label_cols = [col for col in y.columns]
y.index = y.index.str.lower()
addresses = y.index.tolist()

### Split Train

In [85]:
train_idx, test_idx = train_test_split(addresses, test_size=0.3, random_state=42)
y_train = y.loc[train_idx]
y_test = y.loc[test_idx]

## Model

### For txn_static_feature_model

In [86]:
df = load_feature('transaction_feature.csv')
df.index = df.index.str.lower()

# Filter train_idx and test_idx to include only addresses present in df.index
train_idx_filtered = [idx for idx in train_idx if idx in df.index]
test_idx_filtered = [idx for idx in test_idx if idx in df.index]

X_train = df.loc[train_idx_filtered]
X_test = df.loc[test_idx_filtered]
y_train_filtered = y_train.loc[train_idx_filtered]
y_test_fitered = y_test.loc[test_idx_filtered]

#### Result

In [87]:
report_df, _ = report_for_multiple_model_as_same_set(X_train, X_test, y_train_filtered, y_test_fitered)
report_df.sort_values(by='macro avg f1', ascending=False)

[LightGBM] [Info] Number of positive: 12, number of negative: 36
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 225
[LightGBM] [Info] Number of data points in the train set: 48, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.250000 -> initscore=-1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

,micro avg f1,macro avg f1,Mint f1,Leak f1,Limit f1
MultiOutput(RandomForest),0.521739,0.448815,0.222222,0.428571,0.695652
OneVsRest(RandomForest),0.521739,0.448815,0.222222,0.428571,0.695652
OneVsRest(LogisticRegression),0.509804,0.433939,0.181818,0.400000,0.720000
MultiOutput(LogisticRegression),0.509804,0.433939,0.181818,0.400000,0.720000
OneVsRest(GaussianNB),0.491803,0.357771,0.000000,0.363636,0.709677
MultiOutput(GaussianNB),0.491803,0.357771,0.000000,0.363636,0.709677
MultiOutput(MLP),0.238095,0.183333,0.250000,0.000000,0.300000
OneVsRest(MLP),0.238095,0.183333,0.250000,0.000000,0.300000
MultiOutput(SGD),0.206897,0.157407,0.222222,0.000000,0.250000
OneVsRest(SGD),0.206897,0.157407,0.222222,0.000000,0.250000


### For bytecode_static_feature_model

In [88]:
df = load_feature('bytecode_feature.csv')
df.index = df.index.str.lower()

# Filter train_idx and test_idx to include only addresses present in df.index
train_idx_filtered = [idx for idx in train_idx if idx in df.index]
test_idx_filtered = [idx for idx in test_idx if idx in df.index]

X_train = df.loc[train_idx_filtered]
X_test = df.loc[test_idx_filtered]
y_train_filtered = y_train.loc[train_idx_filtered]
y_test_fitered = y_test.loc[test_idx_filtered]

#### Result

In [89]:
report_df, _ = report_for_multiple_model_as_same_set(X_train, X_test, y_train_filtered, y_test_fitered)
report_df.sort_values(by='macro avg f1', ascending=False)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 12, number of negative: 36
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9678
[LightGBM] [Info] Number of data points in the train set: 48, number of used features: 915
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.250000 -> initscore=-1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

,micro avg f1,macro avg f1,Mint f1,Leak f1,Limit f1
MultiOutput(RandomForest),0.564103,0.509402,0.461538,0.4,0.666667
OneVsRest(RandomForest),0.564103,0.509402,0.461538,0.4,0.666667
OneVsRest(AdaBoost),0.486486,0.444444,0.333333,0.4,0.600000
MultiOutput(AdaBoost),0.486486,0.444444,0.333333,0.4,0.600000
OneVsRest(SGD),0.521739,0.379085,0.470588,0.0,0.666667
MultiOutput(SGD),0.521739,0.379085,0.470588,0.0,0.666667
MultiOutput(ExtraTrees),0.512821,0.365079,0.428571,0.0,0.666667
OneVsRest(ExtraTrees),0.512821,0.365079,0.428571,0.0,0.666667
MultiOutput(DecisionTree),0.523810,0.342564,0.307692,0.0,0.720000
OneVsRest(DecisionTree),0.523810,0.342564,0.307692,0.0,0.720000


### For source_code_static_feature_model

In [90]:
df = load_feature('tf_idf.csv')
df.index = df.index.str.lower()

# Filter train_idx and test_idx to include only addresses present in df.index
train_idx_filtered = [idx for idx in train_idx if idx in df.index]
test_idx_filtered = [idx for idx in test_idx if idx in df.index]

X_train = df.loc[train_idx_filtered]
X_test = df.loc[test_idx_filtered]
y_train_filtered = y_train.loc[train_idx_filtered]
y_test_fitered = y_test.loc[test_idx_filtered]

#### Result

In [91]:
report_df, _ = report_for_multiple_model_as_same_set(X_train, X_test, y_train_filtered, y_test_fitered)
report_df.sort_values(by='macro avg f1', ascending=False)

[LightGBM] [Info] Number of positive: 12, number of negative: 36
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4158
[LightGBM] [Info] Number of data points in the train set: 48, number of used features: 355
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.250000 -> initscore=-1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,micro avg f1,macro avg f1,Mint f1,Leak f1,Limit f1
MultiOutput(LogisticRegression),0.604651,0.554386,0.631579,0.400000,0.631579
OneVsRest(LogisticRegression),0.604651,0.554386,0.631579,0.400000,0.631579
MultiOutput(SGD),0.484848,0.440523,0.400000,0.333333,0.588235
OneVsRest(SGD),0.484848,0.440523,0.400000,0.333333,0.588235
OneVsRest(XGBoost),0.437500,0.423529,0.400000,0.400000,0.470588
MultiOutput(XGBoost),0.437500,0.423529,0.400000,0.400000,0.470588
MultiOutput(DecisionTree),0.461538,0.422365,0.461538,0.250000,0.555556
OneVsRest(DecisionTree),0.461538,0.422365,0.461538,0.250000,0.555556
MultiOutput(MLP),0.516129,0.370370,0.444444,0.000000,0.666667
OneVsRest(MLP),0.516129,0.370370,0.444444,0.000000,0.666667


### Time Series

In [92]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Masking
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [93]:
seq_len = 500
feature_dim = 5

X_train, X_test, y_train_filtered, y_test_fitered  = [], [], [], []

for path in tqdm(list(Path(os.path.join(PATH, 'txn')).glob('*.json'))):
    addr = path.stem
    data = json.load(open(path, 'r'))
    txns = sorted(data.get("transaction", []), key=lambda x: int(x.get("timeStamp", 0)))
    seq = [[
        int(tx.get("gas", 0)),
        int(tx.get("gasPrice", 0)),
        int(tx.get("value", 0)),
        int(tx.get("isError", 0)),
        int(tx.get("txreceipt_status", 0))
    ] for tx in txns]

    if len(seq) < seq_len:
        seq += [[0]*feature_dim] * (seq_len - len(seq))
    else:
        seq = seq[:seq_len]

    seq = StandardScaler().fit_transform(seq)

    if addr in train_idx:
        X_train.append(seq)
        y_train_filtered.append(y.loc[addr].tolist())
    elif addr in test_idx:
        X_test.append(seq)
        y_test_fitered.append(y.loc[addr].tolist())

100%|██████████| 69/69 [00:01<00:00, 48.49it/s]


In [94]:
len(X_train), len(y_train_filtered), len(X_test), len(y_test_fitered)

(48, 48, 21, 21)

In [95]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train_filtered = np.array(y_train_filtered)
y_test_fitered = np.array(y_test_fitered)

In [96]:
# === Model ===
model = Sequential([
    Masking(mask_value=0.0, input_shape=(seq_len, feature_dim)),
    GRU(64),
    Dense(3, activation="sigmoid")
])

In [97]:
model.compile(loss=BinaryCrossentropy(), optimizer=Adam(1e-3), metrics=['accuracy'])
model.fit(X_train, y_train_filtered, epochs=30, verbose=1)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 312ms/step - accuracy: 0.3090 - loss: 0.6923
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step - accuracy: 0.4583 - loss: 0.6832
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.3368 - loss: 0.6774
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step - accuracy: 0.3264 - loss: 0.6672
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.3646 - loss: 0.6623
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 301ms/step - accuracy: 0.3646 - loss: 0.6532
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 311ms/step - accuracy: 0.3889 - loss: 0.6467
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 394ms/step - accuracy: 0.4306 - loss: 0.6418
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 377ms/step - accuracy: 0.4340 - loss: 0.6314
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 306ms/step - accuracy: 0.4132 - loss: 0.6235
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 0.4375 - loss: 0.6153
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 280ms/step - accuracy: 0.4583 - lo

In [98]:
# === Predict ===
prob = model.predict(X_test)
thresholds, _ = tune_thresholds(y_test_fitered, prob)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
Label 0: Best threshold = 0.18, Best f1 = 0.5714
Label 1: Best threshold = 0.24, Best f1 = 0.4286
Label 2: Best threshold = 0.32, Best f1 = 0.8148


#### Result

In [99]:
evaluate_multilabel_classification(y_test, prob, label_cols, thresholds)

({'micro_precision': 0.4222222222222222,
  'micro_recall': 0.7916666666666666,
  'micro_f1': 0.5507246376811594,
  'macro_precision': 0.41111111111111115,
  'macro_recall': 0.8055555555555555,
  'macro_f1': 0.5308641975308642,
  'weighted_precision': 0.45,
  'weighted_recall': 0.7916666666666666,
  'weighted_f1': 0.5582010582010581,
  'subset_accuracy': 0.19047619047619047},
               precision    recall  f1-score  support
 Mint           0.400000  1.000000  0.571429      8.0
 Leak           0.300000  0.750000  0.428571      4.0
 Limit          0.533333  0.666667  0.592593     12.0
 micro avg      0.422222  0.791667  0.550725     24.0
 macro avg      0.411111  0.805556  0.530864     24.0
 weighted avg   0.450000  0.791667  0.558201     24.0
 samples avg    0.468254  0.547619  0.465079     24.0)

### Graph

In [100]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCN(nn.Module):
    def __init__(self, in_channels, hidden, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden*2)
        self.conv2 = GCNConv(hidden*2, hidden)
        self.lin = nn.Linear(hidden, out_channels)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

#### Transaction

In [101]:
graph_feature = load_feature('txn_graph_features.csv')
graph_feature.index = graph_feature.index.str.lower()
graphs = pickle.load(open(os.path.join(PATH, 'txn.pkl'), 'rb'))

In [102]:
import torch
from torch_geometric.data import Data

train_dataset, test_dataset = [], []
for i, (address, graph_data) in enumerate(graphs.items()):
    feature = graph_feature.loc[address]
    data = from_networkx(graph_data)

    data.x = torch.tensor(feature, dtype=torch.float32).repeat(data.num_nodes, 1) # Repeat features for each node
    data.y = torch.tensor(y.loc[address][label_cols].values, dtype=torch.float32).unsqueeze(0) # Add a batch dimension

    if address in train_idx:
        train_dataset.append(data)
    elif address in test_idx:
        test_dataset.append(data)

    if i < 10:
        print(data)

Data(edge_index=[2, 1], tx_hash=[1], label=[1], gas=[1], num_nodes=2, x=[2, 6], y=[1, 3])
Data(edge_index=[2, 13], tx_hash=[13], label=[13], gas=[13], num_nodes=14, x=[14, 6], y=[1, 3])
Data(edge_index=[2, 65], tx_hash=[65], label=[65], gas=[65], num_nodes=66, x=[66, 6], y=[1, 3])
Data(edge_index=[2, 56], tx_hash=[56], label=[56], gas=[56], num_nodes=57, x=[57, 6], y=[1, 3])
Data(edge_index=[2, 252], tx_hash=[252], label=[252], gas=[252], num_nodes=253, x=[253, 6], y=[1, 3])
Data(edge_index=[2, 2], tx_hash=[2], label=[2], gas=[2], num_nodes=3, x=[3, 6], y=[1, 3])
Data(edge_index=[2, 5869], tx_hash=[5869], label=[5869], gas=[5869], num_nodes=5870, x=[5870, 6], y=[1, 3])
Data(edge_index=[2, 944], tx_hash=[944], label=[944], gas=[944], num_nodes=945, x=[945, 6], y=[1, 3])
Data(edge_index=[2, 58], tx_hash=[58], label=[58], gas=[58], num_nodes=59, x=[59, 6], y=[1, 3])
Data(edge_index=[2, 2], tx_hash=[2], label=[2], gas=[2], num_nodes=3, x=[3, 6], y=[1, 3])


In [103]:
len(train_dataset), len(test_dataset)

(26, 11)

In [104]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [105]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader

class GCN(nn.Module):
    def __init__(self, in_channels, hidden, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.lin = nn.Linear(hidden, out_channels)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

model = GCN(in_channels=graph_feature.shape[1], hidden=64, out_channels=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss()  # For multi-label classification

In [106]:
for epoch in range(1, 51):  # max 50 epochs
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = loss_fn(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        preds = (torch.sigmoid(out) > 0.5).int().cpu().numpy()
        labels = batch.y.int().cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels)

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    acc = (all_preds == all_labels).mean()

    print(f"Epoch {epoch}, Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

Epoch 1, Loss: 63.4631, Accuracy: 0.6667
Epoch 2, Loss: 28.1353, Accuracy: 0.7949
Epoch 3, Loss: 10.0866, Accuracy: 0.6410
Epoch 4, Loss: 15.6584, Accuracy: 0.6538
Epoch 5, Loss: 37.0788, Accuracy: 0.6410
Epoch 6, Loss: 12.4498, Accuracy: 0.6154
Epoch 7, Loss: 8.9316, Accuracy: 0.6923
Epoch 8, Loss: 16.2597, Accuracy: 0.7564
Epoch 9, Loss: 12.7192, Accuracy: 0.7179
Epoch 10, Loss: 3.7683, Accuracy: 0.7308
Epoch 11, Loss: 8.2867, Accuracy: 0.7179
Epoch 12, Loss: 8.0178, Accuracy: 0.7179
Epoch 13, Loss: 5.8533, Accuracy: 0.8205
Epoch 14, Loss: 4.8556, Accuracy: 0.7949
Epoch 15, Loss: 3.5947, Accuracy: 0.7821
Epoch 16, Loss: 3.9204, Accuracy: 0.7692
Epoch 17, Loss: 3.6680, Accuracy: 0.7692
Epoch 18, Loss: 2.5848, Accuracy: 0.8077
Epoch 19, Loss: 4.2931, Accuracy: 0.7564
Epoch 20, Loss: 3.9962, Accuracy: 0.7821
Epoch 21, Loss: 4.1611, Accuracy: 0.8077
Epoch 22, Loss: 6.0364, Accuracy: 0.7692
Epoch 23, Loss: 3.9888, Accuracy: 0.7821
Epoch 24, Loss: 7.8300, Accuracy: 0.7821
Epoch 25, Loss: 1

In [107]:
model.eval()
y_true, y_probs = [], []
with torch.no_grad():
    for batch in test_loader:
        out = torch.sigmoid(model(batch))
        y_probs.append(out.cpu())
        y_true.append(batch.y.cpu())

y_true = np.vstack(y_true)
y_probs = np.vstack(y_probs)

best_thresholds, _ = tune_thresholds(y_true, y_probs)

Label 0: Best threshold = 0.21, Best f1 = 0.6000
Label 1: Best threshold = 0.16, Best f1 = 0.6667
Label 2: Best threshold = 0.13, Best f1 = 0.7143


##### Result

In [108]:
evaluate_multilabel_classification(y_true, y_probs, label_names=label_cols, threshold=best_thresholds)

({'micro_precision': 0.5,
  'micro_recall': 1.0,
  'micro_f1': 0.6666666666666666,
  'macro_precision': 0.4947089947089947,
  'macro_recall': 1.0,
  'macro_f1': 0.6603174603174603,
  'weighted_precision': 0.5070546737213404,
  'weighted_recall': 1.0,
  'weighted_f1': 0.6708994708994709,
  'subset_accuracy': 0.5454545454545454},
               precision    recall  f1-score  support
 Mint           0.428571  1.000000  0.600000      3.0
 Leak           0.500000  1.000000  0.666667      1.0
 Limit          0.555556  1.000000  0.714286      5.0
 micro avg      0.500000  1.000000  0.666667      9.0
 macro avg      0.494709  1.000000  0.660317      9.0
 weighted avg   0.507055  1.000000  0.670899      9.0
 samples avg    0.409091  0.454545  0.424242      9.0)

#### Control Flow

In [109]:
graph_feature = load_feature('cfg_graph_features.csv')
graph_feature.index = graph_feature.index.str.lower()
graphs = pickle.load(open(os.path.join(PATH, 'cfg.pkl'), 'rb'))

In [110]:
import torch
from torch_geometric.data import Data

train_dataset, test_dataset = [], []
for i, (address, graph_data) in enumerate(graphs.items()):
    feature = graph_feature.loc[address]
    data = from_networkx(graph_data)

    data.x = torch.tensor(feature, dtype=torch.float32).repeat(data.num_nodes, 1) # Repeat features for each node
    data.y = torch.tensor(y.loc[address][label_cols].values, dtype=torch.float32).unsqueeze(0) # Add a batch dimension

    if address in train_idx:
        train_dataset.append(data)
    elif address in test_idx:
        test_dataset.append(data)

    if i < 10:
        print(data)

Data(edge_index=[2, 118], num_nodes=155, x=[155, 6], y=[1, 3])
Data(edge_index=[2, 96], num_nodes=194, x=[194, 6], y=[1, 3])
Data(edge_index=[2, 178], num_nodes=339, x=[339, 6], y=[1, 3])
Data(edge_index=[2, 78], num_nodes=113, x=[113, 6], y=[1, 3])
Data(edge_index=[2, 211], num_nodes=303, x=[303, 6], y=[1, 3])
Data(edge_index=[2, 158], num_nodes=377, x=[377, 6], y=[1, 3])
Data(edge_index=[2, 365], num_nodes=683, x=[683, 6], y=[1, 3])
Data(edge_index=[2, 137], num_nodes=215, x=[215, 6], y=[1, 3])
Data(edge_index=[2, 96], num_nodes=192, x=[192, 6], y=[1, 3])
Data(edge_index=[2, 4], num_nodes=584, x=[584, 6], y=[1, 3])


In [111]:
len(train_dataset), len(test_dataset)

(47, 20)

In [112]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [113]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader

class GCN(nn.Module):
    def __init__(self, in_channels, hidden, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.lin = nn.Linear(hidden, out_channels)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

model = GCN(in_channels=graph_feature.shape[1], hidden=64, out_channels=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss()  # For multi-label classification

In [114]:
for epoch in range(1, 51):  # max 50 epochs
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = loss_fn(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        preds = (torch.sigmoid(out) > 0.5).int().cpu().numpy()
        labels = batch.y.int().cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels)

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    acc = (all_preds == all_labels).mean()

    print(f"Epoch {epoch}, Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

Epoch 1, Loss: 19.5926, Accuracy: 0.6809
Epoch 2, Loss: 10.4809, Accuracy: 0.6879
Epoch 3, Loss: 6.8216, Accuracy: 0.6312
Epoch 4, Loss: 6.3548, Accuracy: 0.6454
Epoch 5, Loss: 6.0185, Accuracy: 0.7730
Epoch 6, Loss: 4.0681, Accuracy: 0.7730
Epoch 7, Loss: 3.9419, Accuracy: 0.6596
Epoch 8, Loss: 4.3268, Accuracy: 0.7234
Epoch 9, Loss: 3.6015, Accuracy: 0.7092
Epoch 10, Loss: 3.4319, Accuracy: 0.6879
Epoch 11, Loss: 4.6782, Accuracy: 0.6738
Epoch 12, Loss: 3.4717, Accuracy: 0.7376
Epoch 13, Loss: 4.0123, Accuracy: 0.7234
Epoch 14, Loss: 3.0898, Accuracy: 0.7872
Epoch 15, Loss: 3.1782, Accuracy: 0.7943
Epoch 16, Loss: 3.4202, Accuracy: 0.7518
Epoch 17, Loss: 2.9155, Accuracy: 0.8085
Epoch 18, Loss: 2.8745, Accuracy: 0.8440
Epoch 19, Loss: 2.8854, Accuracy: 0.7801
Epoch 20, Loss: 2.9376, Accuracy: 0.8085
Epoch 21, Loss: 3.3690, Accuracy: 0.7730
Epoch 22, Loss: 3.8667, Accuracy: 0.7872
Epoch 23, Loss: 3.7555, Accuracy: 0.7021
Epoch 24, Loss: 2.9425, Accuracy: 0.8014
Epoch 25, Loss: 2.9455,

In [115]:
model.eval()
y_true, y_probs = [], []
with torch.no_grad():
    for batch in test_loader:
        out = torch.sigmoid(model(batch))
        y_probs.append(out.cpu())
        y_true.append(batch.y.cpu())

y_true = np.vstack(y_true)
y_probs = np.vstack(y_probs)

best_thresholds, _ = tune_thresholds(y_true, y_probs)

Label 0: Best threshold = 0.18, Best f1 = 0.6667
Label 1: Best threshold = 0.00, Best f1 = 0.3333
Label 2: Best threshold = 0.00, Best f1 = 0.7097


##### Result

In [116]:
evaluate_multilabel_classification(y_true, y_probs, label_names=label_cols, threshold=best_thresholds)

({'micro_precision': 0.4117647058823529,
  'micro_recall': 0.9545454545454546,
  'micro_f1': 0.5753424657534246,
  'macro_precision': 0.4318181818181818,
  'macro_recall': 0.9523809523809524,
  'macro_f1': 0.5698924731182796,
  'weighted_precision': 0.48491735537190084,
  'weighted_recall': 0.9545454545454546,
  'weighted_f1': 0.627565982404692,
  'subset_accuracy': 0.05},
               precision    recall  f1-score  support
 Mint           0.545455  0.857143  0.666667      7.0
 Leak           0.200000  1.000000  0.333333      4.0
 Limit          0.550000  1.000000  0.709677     11.0
 micro avg      0.411765  0.954545  0.575342     22.0
 macro avg      0.431818  0.952381  0.569892     22.0
 weighted avg   0.484917  0.954545  0.627566     22.0
 samples avg    0.375000  0.600000  0.455000     22.0)